In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ugransome-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4144264%2F7172543%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240811%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240811T195155Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D82e758be811c800fce17aea883acd71877a55f84e6db7e2c727ea6dfc54f72c0c9af74cc2a276ed4a327bff689a51d830079c03707eebb70fb940dc1519a671ba5535c83610cea0f8c36f09656daa9413d01b1709ee4fea2f8d773ca807db9751004d05b138b4159edfeb829421e1932dbfa664e296bcb29f8faa15ab056e1f8992a458f3aefe5e125e369efc7bf909fe0c82bbd0b7c1bda055ee8322a87229c69caec8a36d43a9104ddab9a10f831aa0e8abc3ee29ac7d4b6b1776099b4bcbcf5a2c32774fbdead1fd88a16cd831dbaa23009d7fc7b3a7129807ffa76dc5d9160856d2a92bb6663c2fb22e5e3feb025a82e52ad29c247a2834a4bdfe8345179'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**About DataSet**

The UGRansom dataset is a versatile resource for analyzing ransomware and zero-day cyberattacks. It provides timestamps, flags for attack types, protocol data, and network flow details. Additionally, it offers ransomware family classifications, malware insights, and quantifies financial damage in USD and bitcoins.

**The features in the dataset :**

1) Time: Timestamps of network attacks (quantitative).

2) Protocol: Network protocol used (categorical).

3) Flag: Network connection status (categorical).

4) Family: Category of network intrusion (categorical).

5) Clusters: Event clusters or groups (quantitative).

6) SeedAddress: Formatted ransomware attack links (categorical).

7) ExpAddress: Original ransomware attack links (categorical).

8) BTC: Values related to Bitcoin transactions in attacks (numeric).

9) USD: Financial damages in USD caused by attacks (numeric).

10) Netflow Bytes: Bytes transferred in network flow (quantitative).

11) IPaddress: IP addresses associated with network events (categorical).

12) Threats: Nature of threats or intrusions (categorical).

13) Port: Network port number in events (quantitative).

14) Prediction: Target variable indicating predictive model outcomes (anomaly (A), signature (S), and synthetic signature (SS)) (categorical).

In [ ]:
data=pd.read_csv("/kaggle/input/ugransome-dataset/final(2).csv")
data

In [ ]:
data.isnull().sum()

In [ ]:
data.describe().T

In [ ]:
data.info()

In [ ]:
prediction_counts = data['Time'].value_counts()
prediction_counts

**Exploratory Data Analysis (EDA)**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x=data['IPaddress'], data=data)
plt.title('Bar Graph of IPaddress')
plt.show()

In [ ]:
sns.countplot(x=data['Prediction'], data=data)
plt.title('Bar Graph of Prediction')
plt.show()

In [ ]:
sns.countplot(x=data['Protcol'],data=data)
plt.title('Bar Graph of Protcol')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.countplot(x=data['Family'], data=data)
plt.title('Bar Graph of Family')
plt.xticks(rotation=45)
plt.xticks(fontsize=10)
plt.show()

In [ ]:
data['Threats'] = data['Threats'].str.replace('Bonet', 'Botnet')
plt.figure(figsize=(15, 6))
ax = sns.countplot(x=data['Threats'], data=data)
plt.title('Bar Graph of Threats')
plt.xticks(rotation=45)
plt.xticks(fontsize=10)
plt.show()

In [ ]:
sns.countplot(x=data['Flag'],data=data)
plt.title('Bar Graph of Flag')
plt.show()

**Data Preprocessing**

In [ ]:
objList = data.select_dtypes(include = "object").columns
print (objList)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    data[feat] = le.fit_transform(data[feat].astype(str))

print (data.info())

In [ ]:
data

In [ ]:
X=data.drop(['Prediction'],axis=1)
y=data['Prediction']

**Standardization of Features**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

**Train-Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Training Set - X_train shape:", X_train.shape)
print("Testing Set - X_test shape:", X_test.shape)
print("Training Set - y_train shape:", y_train.shape)
print("Testing Set - y_test shape:", y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

**Naive Bayes Model**

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)

nb_accuracy = accuracy_score(nb_pred, y_test)
nb_report = classification_report(nb_pred, y_test)
nb_matrix = confusion_matrix(nb_pred, y_test)
print('Accuracy of Naive Bayes : ', round(nb_accuracy, 3))
print('Classification report of Naive Bayes : \n', nb_report)

In [ ]:
num_wrong = (y_test != nb_pred).sum()

print('Number of incorrect predictions by Naive Bayes:', num_wrong)

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(nb_matrix, annot=True, fmt="d", cmap="inferno", cbar=False,
            xticklabels=["0:A", "1:S", "2:SS"], yticklabels=["0:A", "1:S", "2:SS"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix Heatmap of Naive Bayes")
plt.show()

**SVM Model**

In [ ]:
svr =SVC()
svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)
svr_accuracy = accuracy_score(svr_pred, y_test)
svr_report = classification_report(svr_pred, y_test)
svr_matrix = confusion_matrix(svr_pred, y_test)
print('Accuracy of SVM : ', round(svr_accuracy, 3))
print('Classification report of SVM : \n', svr_report)

In [ ]:
num_wrong = (y_test != svr_pred).sum()

print('Number of incorrect predictions by SVC:', num_wrong)

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(svr_matrix, annot=True, fmt="d", cmap="YlGnBu", cbar=False,
            xticklabels=["0:A", "1:S", "2:SS"], yticklabels=["0:A", "1:S", "2:SS"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix Heatmap of SVM")
plt.show()

**ANN MODEL**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

model = keras.Sequential([
    layers.Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation="selu"),
    layers.Dense(3, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

history = model.fit(X_train, y_train, epochs=50, batch_size=16,
                    callbacks=[early_stopping], validation_split=0.2)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred_class)

In [ ]:
class_report = classification_report(y_test, y_pred_class)
print('Classification Report:')
print(class_report)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
num_wrong = (y_test != y_pred_class).sum()

print('Number of incorrect predictions by ANN:', num_wrong)

In [ ]:
result_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_class})
result_df.head(20)


**RandomForestClassifier Model**

In [ ]:
rfc=RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

rfc_accuracy = accuracy_score(rfc_pred, y_test)
rfc_report = classification_report(rfc_pred, y_test)
rfc_matrix = confusion_matrix(rfc_pred, y_test)
print('Accuracy of RandomForestClassifier : ', round(rfc_accuracy, 3))
print('Classification report of RandomForestClassifier : \n', rfc_report)

In [ ]:
num_wrong = (y_test != rfc_pred).sum()

print('Number of incorrect predictions by RandomForestClassifier:', num_wrong)

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(rfc_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["0:A", "1:S", "2:SS"], yticklabels=["0:A", "1:S", "2:SS"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix Heatmap of RandomForestClassifier")
plt.show()

In [ ]:
feature_importances = rfc.feature_importances_
print("Feature Importances:")
for i, importance in enumerate(feature_importances):
    print(f"Feature {i+1}: {importance}")

In [ ]:
accuracies = [test_acc, nb_accuracy,rfc_accuracy, svr_accuracy]
classifiers = ['ANN','NaiveBayes', 'RandomForest', 'SVM']

plt.figure(figsize=(10, 6))
sns.barplot(x=classifiers, y=accuracies, palette='muted')
plt.xlabel('Classifier')
plt.ylabel('Accuracy')
plt.title('Comparison of Classifier Accuracies')
plt.ylim(0.6, 1.0)
plt.show()